# Introduction to Tensorflow

> Written by Dr Daniel Buscombe, Northern Arizona University

> Part of a series of notebooks for image recognition and classification using deep convolutional neural networks

The notebook is an introduction to the tensorflow library, which is a popular tool for implementing machine (especially deep) learning

## Concepts

TensorFlow is a way of representing computation without actually performing it until asked. In this sense, it is a form of lazy computing, and it allows for some great improvements to the running of code:

* Faster computation of complex variables
* Distributed computation across multiple systems, including GPUs.
* Reduced redundency in some computations

Let’s have a look at this in action. First, a very basic python script:

In [ ]:
x = 35
y = x + 5
print(y)

a TensorFlow equivalent:

In [ ]:
import  tensorflow  as tf

x = tf.constant(35, name='x')
y = tf.Variable(x + 5, name='y')

print(y)

The output isn't 40. Why? 

our program actually does something quite different to the previous one. The code here does the following:

* Import the tensorflow module and call it tf
* Create a constant value called x, and give it the numerical value 35
* Create a Variable called y, and define it as being the equation x + 5
* Print out the equation object for y

 In effect, y is “when this variable is computed, take the value of x (as it is then) and add 5 to it”. 
    
The computation of the value of y is never actually performed. Let's change that:

In [ ]:
model = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(model)
    print(session.run(y))

What did this do?
* Initialize the variables with global_variables_initializer
* Create a session for computing the values
* Run the model 
* Run just the variable y and print out its current value

when we initialize the variables, a graph is created of the dependencies between the variables. In this case, the variable y depends on the variable x, and that value is transformed by adding 5 to it. Keep in mind that this value isn’t computed until the last step, as up until then, only equations and relations are computed.

## Working with imagery

In [ ]:
from imageio import imread
import matplotlib.pyplot as plt
import s3fs
fs = s3fs.S3FileSystem(anon=True)

In [ ]:
with fs.open('cdi-workshop/imrecog_data/plankton/train/amphipods/10494.jpg', 'rb') as f:
    image = imread(f, 'jpg') 
plt.figure(figsize=(10,10))
plt.imshow(image, cmap='gray');    

### Geometric transformations

The first transformation we will perform is a transpose, turning the image 90 degrees counter-clockwise

In [ ]:
# Create a Tensorflow Variable
x = tf.Variable(image, name='x')

model = tf.global_variables_initializer() 

with tf.Session() as session:
    x = tf.transpose(x)
    session.run(model)
    result = session.run(x)
        
plt.figure(figsize=(10,10))        
plt.imshow(result, cmap='gray')

## Placeholders and 'feed dictionaries'

So far we have used variables to manage our data, but there is a more basic structure, the placeholder. 

A placeholder is simply a variable that we will assign data to at a later date. It allows us to create our operations and build our computation graph, without needing the data. 

In Tensorflow terminology, we then feed data into the graph through these placeholders.

We create a placeholder called x, i.e. a place in memory where we will store value later on. The 'None' on this line allows x to be any length

In [ ]:
x = tf.placeholder("float", None)

Then, we create a Tensor called, which is the operation of multiplying x by 2. Note that we haven’t defined any initial values for x yet.

In [ ]:
y = x * 2

We now have an operation (y) defined, and can now run it in a session. 

We create a session object, and then run just the y variable. This means that if we defined a much larger graph of operations, we can run just a small segment of the graph. This subgraph evaluation is actually a bit selling point of TensorFlow, and one that isn’t present in many other libraries that do similar things.

Running y requires knowledge about the values of x. We define these inside the feed_dict argument to run. We state here that the values of x are [1, 2, 3]. We run y, giving us the result of [2, 4, 6].

In [ ]:
with tf.Session() as session:
    result = session.run(y, feed_dict={x: [1, 2, 3]})
    print(result)

when we define the values of x in the feed_dict we can have any number of values. The code should still work, and give the same answer, but now it will also work with any number of values in feed_dict.

In [ ]:
with tf.Session() as session:
    x_data = [[1, 2, 3],
              [4, 5, 6],]
    result = session.run(y, feed_dict={x: x_data})
    print(result)

We can extend this to take an arbitrary number of None dimensions. In this example, we load up the image from our last lesson, then create a placeholder that stores a slice of that image. 

We then use TensorFlow’s slice method to take a subsegment out of the image to operate on.

We'll take a 20 pixel slice (in vertical) starting at x=10, y=10

In [ ]:
image_placeholder = tf.placeholder("uint8", [None, None])
slice = tf.slice(image_placeholder, [10, 10], [20, -1])

with tf.Session() as session:
    result = session.run(slice, feed_dict={image_placeholder: image})
    print(result.shape)

plt.imshow(result, cmap='gray')

For a RGB image:

In [ ]:
with fs.open('cdi-workshop/imrecog_data/Imagenet/train/butte/n09230202_10020.JPEG', 'rb') as f:
    image = imread(f, 'jpg') 
plt.figure(figsize=(10,10))
plt.imshow(image);  

In [ ]:
image_placeholder = tf.placeholder("uint8", [None, None, None])
slice = tf.slice(image_placeholder, [10, 10, 0], [200, -1, -1])

with tf.Session() as session:
    result = session.run(slice, feed_dict={image_placeholder: image})
    print(result.shape)

plt.imshow(result)

In the above -1 means "all". We can replace that with an actual length in the horizontal dimension

In [ ]:
image_placeholder = tf.placeholder("uint8", [None, None, None])
slice = tf.slice(image_placeholder, [10, 10, 0], [200, 100, -1])

with tf.Session() as session:
    result = session.run(slice, feed_dict={image_placeholder: image})
    print(result.shape)

plt.imshow(result)